In [100]:
%matplotlib inline

from os import listdir

import torch
import h5py, os
import numpy as np
from matplotlib import pyplot as plt
from functions import transforms as T
from functions.subsample import MaskFunc
from skimage.measure import compare_ssim 

# Functions

In [101]:
def show_slices(data, slice_nums, cmap=None): # visualisation
    fig = plt.figure(figsize=(15,10))
    for i, num in enumerate(slice_nums):
        plt.subplot(1, len(slice_nums), i + 1)
        plt.imshow(data[num], cmap=cmap)
        plt.axis('off')
        
def ssim(gt, pred):
    """ Compute Structural Similarity Index Metric (SSIM). """
    return compare_ssim(
        gt.transpose(1, 2, 0), pred.transpose(1, 2, 0), multichannel=True, data_range=gt.max()
    )

def save_reconstructions(reconstructions, out_dir):
    """
    Saves the reconstructions from a model into h5 files that is appropriate for submission
    to the leaderboard.
    Args:
        reconstructions (dict[str, np.array]): A dictionary mapping input filenames to
            corresponding reconstructions (of shape num_slices x height x width).
        out_dir (pathlib.Path): Path to the output directory where the reconstructions
            should be saved.
    """
    for fname, recons in reconstructions.items():
        subject_path = os.path.join(out_dir, fname)
        print(subject_path)
        with h5py.File(subject_path, 'w') as f:
            f.create_dataset('reconstruction', data=recons)

# Load Training Files

In [102]:
train_folder_path = 'C:\\Users\\Jonat\\Documents\\NueralData\\Train\\'

train_file = os.listdir(train_folder_path)

# Generate/Save 4AF and 8AF

In [107]:
for file in train_file:
    train_file_path = train_folder_path + file
    with h5py.File(train_file_path,  "r") as hf:
        volume_kspace = hf['kspace'][()]
    
    volume_kspace2 = T.to_tensor(volume_kspace)      # Convert from numpy array to pytorch tensor
    volume_image = T.ifft2(volume_kspace2)            # Apply Inverse Fourier Transform to get the complex image
    volume_image_abs = T.complex_abs(volume_image)   # Compute absolute value to get a real image

    mask_func0 = MaskFunc(center_fractions=[0.08], accelerations=[4])  # Create the mask function object
    mask_func1 = MaskFunc(center_fractions=[0.04], accelerations=[8])  # Create the mask function object

    shape = np.array(volume_kspace2.shape)

    mask0 = mask_func0(shape, seed=0) # use seed here to exclude randomness  
    mask1 = mask_func1(shape, seed=0)

    masked_kspace0 = torch.where(mask0 == 0, torch.Tensor([0]), volume_kspace2) # masked kspace data with AF=4
    masked_kspace1 = torch.where(mask1 == 0, torch.Tensor([0]), volume_kspace2) # masked kspace data with AF=8

    S_Num, Ny, Nx, _ = volume_kspace2.shape
    masks0 = mask0.repeat(S_Num, Ny, 1, 1).squeeze() # masks when AF=4
    masks1 = mask1.repeat(S_Num, Ny, 1, 1).squeeze() # masks when AF=8

    sampled_image0 = T.ifft2(masked_kspace0)             # Apply Inverse Fourier Transform to get the complex image
    sampled_image_abs0 = T.complex_abs(sampled_image0)   # Compute absolute value to get a real image

    sampled_image1 = T.ifft2(masked_kspace1)             # Apply Inverse Fourier Transform to get the complex image
    sampled_image_abs1 = T.complex_abs(sampled_image1)   # Compute absolute value to get a real image

    # crop the images to the central 320x320 pixel region, as evaluation will be made based on cropped images
    cropped_gt = T.center_crop(volume_image_abs, [320, 320])
    cropped_4af = T.center_crop(sampled_image_abs0, [320, 320])
    cropped_8af = T.center_crop(sampled_image_abs1, [320, 320]) 
    
    fname0 = 'GrounTruth' + file 
    fname1 = '4AF' + file 
    fname2 = '8AF' + file 
    reconstructions = {fname0: cropped_gt.numpy(), fname1: cropped_4af.numpy(), fname2: cropped_8af.numpy()}
    out_dir = 'C:\\Users\\Jonat\\Documents\\NueralData\\Saved' # where you want to save your result. 
    if not (os.path.exists(out_dir)): os.makedirs(out_dir)
    save_reconstructions(reconstructions, out_dir)

    

C:\Users\Jonat\Documents\NueralData\Saved\GrounTruthfile1000000.h5
C:\Users\Jonat\Documents\NueralData\Saved\4AFfile1000000.h5
C:\Users\Jonat\Documents\NueralData\Saved\8AFfile1000000.h5
C:\Users\Jonat\Documents\NueralData\Saved\GrounTruthfile1000007.h5
C:\Users\Jonat\Documents\NueralData\Saved\4AFfile1000007.h5
C:\Users\Jonat\Documents\NueralData\Saved\8AFfile1000007.h5
C:\Users\Jonat\Documents\NueralData\Saved\GrounTruthfile1000017.h5
C:\Users\Jonat\Documents\NueralData\Saved\4AFfile1000017.h5
C:\Users\Jonat\Documents\NueralData\Saved\8AFfile1000017.h5
C:\Users\Jonat\Documents\NueralData\Saved\GrounTruthfile1000026.h5
C:\Users\Jonat\Documents\NueralData\Saved\4AFfile1000026.h5
C:\Users\Jonat\Documents\NueralData\Saved\8AFfile1000026.h5
C:\Users\Jonat\Documents\NueralData\Saved\GrounTruthfile1000031.h5
C:\Users\Jonat\Documents\NueralData\Saved\4AFfile1000031.h5
C:\Users\Jonat\Documents\NueralData\Saved\8AFfile1000031.h5
C:\Users\Jonat\Documents\NueralData\Saved\GrounTruthfile1000033.h

C:\Users\Jonat\Documents\NueralData\Saved\8AFfile1000432.h5
C:\Users\Jonat\Documents\NueralData\Saved\GrounTruthfile1000464.h5
C:\Users\Jonat\Documents\NueralData\Saved\4AFfile1000464.h5
C:\Users\Jonat\Documents\NueralData\Saved\8AFfile1000464.h5
C:\Users\Jonat\Documents\NueralData\Saved\GrounTruthfile1000476.h5
C:\Users\Jonat\Documents\NueralData\Saved\4AFfile1000476.h5
C:\Users\Jonat\Documents\NueralData\Saved\8AFfile1000476.h5
C:\Users\Jonat\Documents\NueralData\Saved\GrounTruthfile1000477.h5
C:\Users\Jonat\Documents\NueralData\Saved\4AFfile1000477.h5
C:\Users\Jonat\Documents\NueralData\Saved\8AFfile1000477.h5
C:\Users\Jonat\Documents\NueralData\Saved\GrounTruthfile1000480.h5
C:\Users\Jonat\Documents\NueralData\Saved\4AFfile1000480.h5
C:\Users\Jonat\Documents\NueralData\Saved\8AFfile1000480.h5
C:\Users\Jonat\Documents\NueralData\Saved\GrounTruthfile1000496.h5
C:\Users\Jonat\Documents\NueralData\Saved\4AFfile1000496.h5
C:\Users\Jonat\Documents\NueralData\Saved\8AFfile1000496.h5
C:\Us